In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np
from novograd import Novograd
from pred_on_the_fly import On_the_fly_Coughing , visualize_spectrogram

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
shirnked_labels={0: 'healthy', 1: 'unknow_rep_illness', 2: 'covid_positive'}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
csv_path='test.csv'
positive_with_covid=On_the_fly_Coughing(csv_path,[3,4,5,6] )
healthy=On_the_fly_Coughing(csv_path,[0])

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu

covid_excl_asy_loader = torch.utils.data.DataLoader(positive_with_covid, batch_size = 1, shuffle = True, **kwargs)
healthy_loader = torch.utils.data.DataLoader(healthy, batch_size = 1, shuffle = True, **kwargs)


In [4]:

covid_pos_wav_loc, covid_pos_in_data, covid_lb= iter(covid_excl_asy_loader).next()
healthy_wav_loc, healthy_in_data, healthy_lb= iter(healthy_loader).next()


print(covid_pos_wav_loc[0], covid_lb)
print(healthy_wav_loc[0], healthy_lb)

wav file location  /workspace/nemo/covid/data/Fx2gbIoPlJXrK5OD8FMeepOGlLa2/cough-heavy.wav
label  covid_positive
wav file location  /workspace/nemo/covid/data/95XA3QmnULQ1KPlcdw4RDDKnBhH2/cough-heavy.wav
label  healthy
/workspace/nemo/covid/data/Fx2gbIoPlJXrK5OD8FMeepOGlLa2/cough-heavy.wav tensor([2])
/workspace/nemo/covid/data/95XA3QmnULQ1KPlcdw4RDDKnBhH2/cough-heavy.wav tensor([0])


In [5]:
num2labels={0: 'healthy', 1: 'resp_illness_not_identified', 2: 'no_resp_illness_exposed', 3: 'recovered_full', 4: 'positive_mild', 5: 'positive_asymp', 6: 'positive_moderate'}


In [5]:
#healthy_np=healthy_in_data.detach().cpu().numpy()
#np.save('healthy.npy', healthy_np)
#healthy_np.shape 


(1, 1, 32000)

In [6]:
covid_in_data_np=covid_pos_in_data.detach().cpu().numpy()
np.save('covid.npy', covid_in_data_np)
covid_in_data_np.shape


(1, 1, 32000)

In [7]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("./saved_model/relabelled_covid.onnx")

# Check that the IR is well formed
onnx.checker.check_model(onnx_model)

# Print a human readable representation of the graph
onnx.helper.printable_graph(onnx_model.graph)

"graph torch-jit-export (\n  %input[FLOAT, 1x1x32000]\n) initializers (\n  %54[FLOAT, 512x3]\n  %learned_0[FLOAT, 128x1x80]\n  %learned_1[FLOAT, 128]\n  %learned_10[FLOAT, 128]\n  %learned_11[FLOAT, 128]\n  %learned_12[FLOAT, 128]\n  %learned_14[FLOAT, 256x128x3]\n  %learned_15[FLOAT, 256]\n  %learned_16[FLOAT, 256]\n  %learned_17[FLOAT, 256]\n  %learned_18[FLOAT, 256]\n  %learned_19[FLOAT, 256]\n  %learned_2[FLOAT, 128]\n  %learned_21[FLOAT, 512x256x3]\n  %learned_22[FLOAT, 512]\n  %learned_23[FLOAT, 512]\n  %learned_24[FLOAT, 512]\n  %learned_25[FLOAT, 512]\n  %learned_26[FLOAT, 512]\n  %learned_29[FLOAT, 3]\n  %learned_3[FLOAT, 128]\n  %learned_4[FLOAT, 128]\n  %learned_5[FLOAT, 128]\n  %learned_7[FLOAT, 128x128x3]\n  %learned_8[FLOAT, 128]\n  %learned_9[FLOAT, 128]\n) {\n  %31 = Conv[dilations = [1], group = 1, kernel_shape = [80], pads = [0, 0], strides = [4]](%input, %learned_0, %learned_1)\n  %32 = BatchNormalization[epsilon = 9.99999974737875e-06, momentum = 0.899999976158142](

In [8]:

covid_pos_wav_loc, covid_pos_in_data, covid_lb= iter(covid_excl_asy_loader).next()
healthy_wav_loc, healthy_in_data, healthy_lb= iter(healthy_loader).next()


wav file location  /workspace/nemo/covid/data/YRiIODBG3RXPbDttwztMhsXg1rv2/cough-heavy.wav
label  covid_positive
wav file location  /workspace/nemo/covid/data/zPFIJ3JDo7P5qIIdKvELq76kYi02/cough-heavy.wav
label  healthy


In [10]:
import IPython.display as ipd
def listen2audio(wav_loc, lb):
    audio_sample=wav_loc[0]
    print("originally labelled by expert as {}".format(shirnked_labels[lb.item()])  )
    return ipd.Audio(audio_sample) # load a local WAV file

listen2audio(covid_pos_wav_loc, covid_lb)

originally labelled by expert as covid_positive


In [9]:
import onnxruntime as ort

def onnx_prediction(onnx_model, input_numpy ):
    
    ort_session = ort.InferenceSession(onnx_model)
    outputs = ort_session.run(None, {'input': input_numpy})
    print(outputs[0][0])
    pred=np.argmax(outputs[0][0], axis=-1)
    out_pred=shirnked_labels[pred[0]]
    return out_pred

output=onnx_prediction('./saved_model/relabelled_covid.onnx', covid_pos_in_data.detach().cpu().numpy())
print("onnx runtime prediction :{} | actual label :{}".format(output, shirnked_labels[covid_lb.item()]))


[[-3.715228   -3.0814688  -0.07283057]]
onnx runtime prediction :covid_positive | actual label :covid_positive


In [12]:
output=onnx_prediction('./saved_model/relabelled_covid.onnx', healthy_in_data.detach().cpu().numpy())
print("onnx runtime prediction :{} | actual label :{}".format(output, shirnked_labels[healthy_lb.item()]))


[[-7.152555e-07 -1.409974e+01 -1.717325e+01]]
onnx runtime prediction :healthy | actual label :healthy
